In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
###### Pip Install ######

# !pip install -qq torch==1.8.0+cu111 torchvision==0.9.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install -qq wandb timm==0.3.2 six tensorboardX

!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install torchtext==0.9.1
!pip install -qq wandb timm==0.3.2 six tensorboardX

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |█████████████▌                  | 834.1 MB 76.2 MB/s eta 0:00:16tcmalloc: large alloc 1147494400 bytes == 0x3c7c000 @  0x7fbf4feaf615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████               | 1055.7 MB 97.9 MB/s eta 0:00:10tcmalloc: large alloc 1434370048 bytes == 0x482d2000 @  0x7fbf4feaf615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████▋          | 1336.2 MB 1.2 MB/s eta 0:09:18tcmalloc: large alloc 1792966656 b

In [3]:
###### Check for GPU availability ######

import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.8.1+cu111 (Tesla P100-PCIE-16GB)


In [4]:
###### Libraries ######

import os
import shutil

In [5]:
###### Helpers ######

def save_model_ckpt(dir):
  model_ckpt_old_dir = os.path.join(os.getcwd(), 'model_ckpt')
  model_ckpt_new_dir = os.path.join(dir, 'model_ckpt')

  try:
    os.makedirs(model_ckpt_new_dir)
  except FileExistsError:
    pass

  for file_name in os.listdir(model_ckpt_old_dir):
    file_old_dir = os.path.join(model_ckpt_old_dir, file_name)
    file_new_dir = os.path.join(model_ckpt_new_dir, file_name)
    shutil.copy2(file_old_dir, file_new_dir)
  print(f'ConvNeXt Model Checkpoints saved in {model_ckpt_new_dir}')

In [6]:
###### Directories ######

dir = '/content/drive/MyDrive/a2/Image-Classification-ConvNeXt' # Change your working directory here
train_dir = os.path.join(dir, 'train')
val_dir = os.path.join(dir, 'val')

In [7]:
##### Facebook ConvNeXt #####
# !rm -rf ConvNeXt
!git clone https://github.com/facebookresearch/ConvNeXt

Cloning into 'ConvNeXt'...
remote: Enumerating objects: 246, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 246 (delta 54), reused 42 (delta 42), pack-reused 162
Receiving objects: 100% (246/246), 71.38 KiB | 1.78 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [8]:
##### ConvNeXt-T Pretrained Weights #####

!wget -P ConvNeXt https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth

--2022-05-05 16:56:08--  https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114414741 (109M) [binary/octet-stream]
Saving to: ‘ConvNeXt/convnext_tiny_1k_224_ema.pth’

convnext_tiny_1k_22 100%[===================>] 109.11M  18.4MB/s    in 7.1s    

2022-05-05 16:56:16 (15.3 MB/s) - ‘ConvNeXt/convnext_tiny_1k_224_ema.pth’ saved [114414741/114414741]



In [9]:
##### Weights & Biases #####

import wandb
# wandb.login()
!wandb login 1752d2f0785658c328f0ed6d6c0bc7debd495d9c
wandb.init(project="CS424_A2")

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: lesterye. Use `wandb login --relogin` to force relogin


In [10]:
##### Train ##### 
##### ./ConvNeXt/convnext_tiny_1k_224_ema.pth ######

!python ConvNeXt/main.py --epochs 50 \
                         --model convnext_tiny \
                         --data_set image_folder \
                         --data_path {train_dir} \
                         --eval_data_path {val_dir} \
                         --nb_classes 92 \
                         --num_workers 8 \
                         --warmup_epochs 0 \
                         --save_ckpt true \
                         --output_dir model_ckpt \
                         --finetune ./ConvNeXt/convnext_tiny_1k_224_ema.pth \
                         --cutmix 0 \
                         --mixup 0 --lr 4e-4 \
                         --enable_wandb true --wandb_ckpt true \
                         --batch_size 64

Not using distributed mode
Namespace(aa='rand-m9-mstd0.5-inc1', auto_resume=True, batch_size=64, clip_grad=None, color_jitter=0.4, crop_pct=None, cutmix=0.0, cutmix_minmax=None, data_path='/content/drive/MyDrive/a2/Image-Classification-ConvNeXt/train', data_set='image_folder', device='cuda', disable_eval=False, dist_eval=True, dist_on_itp=False, dist_url='env://', distributed=False, drop_path=0, enable_wandb=True, epochs=50, eval=False, eval_data_path='/content/drive/MyDrive/a2/Image-Classification-ConvNeXt/val', finetune='./ConvNeXt/convnext_tiny_1k_224_ema.pth', head_init_scale=1.0, imagenet_default_mean_and_std=True, input_size=224, layer_decay=1.0, layer_scale_init_value=1e-06, local_rank=-1, log_dir=None, lr=0.0004, min_lr=1e-06, mixup=0.0, mixup_mode='batch', mixup_prob=1.0, mixup_switch_prob=0.5, model='convnext_tiny', model_ema=False, model_ema_decay=0.9999, model_ema_eval=False, model_ema_force_cpu=False, model_key='model|module', model_prefix='', momentum=0.9, nb_classes=92, 

In [11]:
dir = '/content/drive/MyDrive/a2/Image-Classification-ConvNeXt'

In [12]:
##### Save model_checkpoint in Local Directory #####

save_model_ckpt(dir) # specify parent directory to save model_ckpt

ConvNeXt Model Checkpoints saved in /content/drive/MyDrive/a2/Image-Classification-ConvNeXt/model_ckpt


In [13]:
!ls /content/drive/MyDrive/a2/Image-Classification-ConvNeXt/model_ckpt

checkpoint-47.pth  checkpoint-49.pth	log.txt
checkpoint-48.pth  checkpoint-best.pth


In [ ]:
from time import time
import multiprocessing as mpfor num_workers in range(2, mp.cpu_count(), 2):  
    train_loader = DataLoader(train_reader,shuffle=True,num_workers=num_workers,batch_size=64,pin_memory=True)
start = time()
    for epoch in range(1, 3):
        for i, data in enumerate(train_loader, 0):
            pass
    end = time()
    print("Finish with:{} second, num_workers={}".format(end - start, num_workers))